In [ ]:
import numpy as np
import sympy as sm
from geoalg import ProjectiveGeometry3D

np.random.seed(0)

In [ ]:
A = ProjectiveGeometry3D.create()
e0, e1, e2, e3 = A.blades[1]
e01, e02, e03, e12, e13, e23 = A.blades[2]
e012, e013, e023, e123 = A.blades[3]
e0123, = A.blades[4]
A.e0, A.e1, A.e2, A.e3 = e0, e1, e2, e3

In [ ]:
# res = A.codegen_binary_op(lambda a, b: a * b)
# print(res)

In [ ]:
# res = A.codegen_binary_op(A.sandwich)
# print(res)

In [ ]:
def random_blade(grade, maxint=1):
    return sum(np.random.randint(-maxint, maxint) * b for b in A.blades[grade])

def random_multivec(maxint=1):
    return sum(random_blade(i, maxint=maxint) for i in range(len(A.blades)))

In [ ]:
def test(a, b, func1, func2):
    c1 = func1(a, b)
    c2 = func2(a, b)
    if c1 != c2:
        raise RuntimeError('Products not equivalent.\na: {}\nb: {}\nc1: {}\nc2: {}'.format(a, b, c1, c2))
    assert c1 == c2

# Teaching outline

In [ ]:
e1, e2, e3 = sm.symbols('e1 e2 e3', commutative=False)

In [ ]:
vec = e1 + 2 * e2 - 3 * e3
vec

In [ ]:
e1 * e2

Define simpliciation rules:  

a) The product of two bases is one (think dot product): 
a) $e_i e_i = 1$ (contraction)  
b) $e_i e_j = -e_j e_i$  (anti-commutation)

In [ ]:
# Substitutions for R3 that match the above rules
subs = {
    e1**2: 1,
    e2**2: 1,
    e3**2: 1,
    e2*e1: -e1*e2,
    e3*e1: -e1*e3,
    e3*e2: -e2*e3
}

In [ ]:
def simp(a):
    """ Apply simplifications until no changes occur. """
    while True:
        b = sm.expand(a).subs(subs)
        if b == a:
            return b
        a = b

In [ ]:
simp(e1 * e1)

In [ ]:
simp(e1 * e2)

In [ ]:
simp(e2 * e1)

In [ ]:
simp(e2 * e1 * e2)

In [ ]:
simp((e1 + 2*e2 + 3*e3)**2)

In [ ]:
simp((e1 + 2*e2) * (3 * e2))

In [ ]:
# Scalars are dual to pseudoscalar
# Planes are the same as vectors (vector is 0 fourth component, plane through origin)
# Points are dual to planes
# Lines are intermediate, dual to themselves
# Points with zero e0 component are ideal points (vectors / direction). Dual to planes through the origin.
# Lines through the origin have no e0*ei components, are dual to ideal lines
# Complex numbers - square root of -1

In [ ]:
# Rotations are about planes not vectors
#  - think 2D case, 4D case. way more intuitive. no line out of page
# Specifying lines as point + direction is annoying
# Specifying planes as normal vector and offset separately is annoying
# Quaternion math, derivation from reflection
# Identical implementation of 2D and 3D, including pose transforms
# Treatment of translations and rotations
# Lie algebra derivation
# Autodiff with pseudoscalar

In [ ]:
# lie groups = confusing
# quaternion math = confusing
# handling of infinity in projective geometry = confusing
# cross products = confusing

# a * b = |a||b| * (cos(ang) + sin(ang) * B)
# where B is a bivector of two orthogonal unit vectors in the plane of a and b
# for unit a, b
# a * b = cos(ang) + sin(ang) * B

In [ ]:
# Start with geometric product idea
# Show that it is a closed ring (closed group, addition and multiplication)
# Associative but not commutative
# Reason about what different grades mean
# General product has multiple resulting grades
# Call lowest one inner product and highest one outer product
# rotors / screws
# Notion of duality
# Use in quantum mechanics

In [ ]:
# A duality that is an involution (inverse to itself) is called polarity, and can be done with a * I**-1

In [ ]:
# Euclidian R(2, 0, 0)
# 0) scalar
# 1) vector (oriented length)
# 2) PSS

# Euclidian R(3, 0, 0)
# 0) scalar
# 1) vector (oriented length)
# 2) area
# 3) PSS

# Dual Euclidian R*(2, 0, 0)
# 0) scalar
# 1) vector (oriented length)
# 2) PSS

# Dual Euclidian R*(3, 0, 0)
# 0) scalar
# 1) oriented area
# 2) oriented length (vector)
# 3) PSS

# Homogenous Euclidian R(2, 0, 1)
# 0) scalar
# 1) point
# 2) line
# 3) PSS

# Homogenous Euclidian R(3, 0, 1)
# 0) scalar
# 1) point
# 2) line
# 3) plane
# 4) PSS

# Projective Euclidian R*(2, 0, 1)
# 0) scalar
# 1) line
# 2) point
# 3) PSS

# Projective Euclidian R*(3, 0, 1)
# 0) scalar
# 1) plane
# 2) line
# 3) point
# 4) PSS

In [ ]:
l = A.outer(e1 + 3*e3 + e0, 3*e3 + e0)
l

In [ ]:
A.dual(l)

In [ ]:
# Create two points
P1 = A.point(1, 0, 3)
P2 = A.point(0, 0, 3)
P1

In [ ]:
# Line between two points
l = A.join(P1, P2)
l

In [ ]:
# Create plane
p = A.plane(1, 1, 0, 1)
p

In [ ]:
p / A.product(p, p)

In [ ]:
# Line orthogonal to plane p through point P
A.inner(p, P1)

In [ ]:
# Plane orthogonal to plane p through line l
A.inner(p, l)

In [ ]:
A.point(0, 0, 0)

In [ ]:
A.inner(A.plane(1, 2, 3, 0), A.point(0, 0, 0))

In [ ]:
# Create a line
l = A.join(A.point(0, 0, 0), A.point(0, 0, 1))
l

In [ ]:
# Create rotor along the line
rotor = A._rotator(l, np.deg2rad(45))
rotor

In [ ]:
# Rotate point
P1

In [ ]:
A.sandwich(rotor, p)

In [ ]:
translator = A.translator(2.5, 0, 0)
translator

In [ ]:
P = A.point(*sm.symbols('P_x P_y P_z'))
P

In [ ]:
t = A.translator(*sm.symbols('t_x t_y t_z'))
t

In [ ]:
A.sandwich(t, P)

In [ ]:
def rotator(self, x, y, z):
    return self.simp(self._rotator(self.line(0, 0, 0, x, y, z), 1))
rotator(A, 0, 0, np.deg2rad(30.)).evalf()

In [ ]:
axis = sm.symbols('a_x a_y a_z')
angle = sm.Symbol('theta')
rotor = A.simp(A._rotator(A.line(0, 0, 0, axis[0], axis[1], axis[2]), angle))
rotor

In [ ]:
line = A.line(0, 0, 0, axis[0], axis[1], axis[2])

In [ ]:
A.normalized(line)

In [ ]:
R = A.axis_angle(axis, angle)
R

In [ ]:
Rt = A.product(R, t)
Rt

In [ ]:
A.sandwich(Rt, P)

In [ ]:
rot = A.axis_angle([0, -1, 0], np.deg2rad(45))
rot

In [ ]:
A.sandwich(rot, A.point(1, 0, 0))

In [ ]:
p = A.plane(1, 2, 3, 1)
p

In [ ]:
p2 = A.plane(-1, 2, 1, 1)
p2

In [ ]:
line = A.outer(p, p2)
line

In [ ]:
plane = A.point(0, 0, 1)
plane

In [ ]:
A.simp(A.inner(plane, line) / -4)

In [ ]:
u = A.normalized(e1 + 2 * e2 + e3)
v = A.normalized(4 * e1 + e2)
u, v

In [ ]:
rotor = A.product(u, v)
rotor

In [ ]:
direction = 5 * e1
direction

In [ ]:
A.sandwich(rotor, 10*e1)

In [ ]:
A.simp(-e1 * (sm.Symbol('x') * e1 + sm.Symbol('y') * e2 + sm.Symbol('z') * e3))

In [ ]:
A.sandwich(e1, direction)

In [ ]:
A.sandwich(e1 * e2 * e3, direction)

In [ ]:
A.dual(5*e3)

In [ ]:
A.sandwich(e3, 5 * e3 + e0)

In [ ]:
A.sandwich(e3, 5)

In [ ]:
A.sandwich(e3, 5 * e012)

In [ ]:
ops = {
    'sum': 2,
    'product': 2,
    'inner': 2,
    'outer': 2,
    'dual': 1,
    'reverse': 1,
    'polarity': 1,
    'regressive': 2,
    'commutator': 2,
    'sandwich': 2
}

In [ ]:
# for name, num_args in sorted(ops.items(), key=lambda v: v[0]):
#     print(A.codegen_op(getattr(A, name), num_args, name))
#     print('')

In [ ]:
e3 * (5 * e3) * A.reverse(e3)

In [ ]:
A.reverse(e3)

In [ ]:
-A.sandwich(e3, e2)

In [ ]:
A.simp(e3 * e2 * A.reverse(e3))